In [60]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

pickle_in = open("match_dataframe_processed.pickle","rb")
match_dataframe = pickle.load(pickle_in)

display(match_dataframe)

#I see a lot of -1's in columns GoldSpent and GoldRemaining. 
#These are the missing data points which should be filled out. 
#I want to analyze the gold per minute, gold spent and gold remaining columns separately, 
#so I will create a new dataframe for them.
match_dataframe_gold_analysis = match_dataframe[['Duration', 'GoldPerMin', 'GoldSpent', 'GoldRemaining']].copy()

#First, create a column with Duration in minutes, in decimal form 
match_dataframe_gold_analysis['DurationMins'] = match_dataframe_gold_analysis['Duration'].map(lambda x: round((x.total_seconds()/60), 2))

#To begin with, let me calculate total gold by the data given gold spent and remaining
#and another row which calculates gold per minute times match duration
match_dataframe_gold_analysis['TotalGoldSum'] = match_dataframe_gold_analysis['GoldSpent'] + match_dataframe_gold_analysis['GoldRemaining']
match_dataframe_gold_analysis['TotalGoldFromGPM'] = (match_dataframe_gold_analysis['GoldPerMin'] * match_dataframe_gold_analysis['DurationMins']).round().astype(int)
match_dataframe_gold_analysis['GoldDifference'] = match_dataframe_gold_analysis['TotalGoldSum'] - match_dataframe_gold_analysis['TotalGoldFromGPM']

#Check result
display(match_dataframe_gold_analysis)

#Remove rows with the -1's to get a better undestanding of the difference
match_dataframe_gold_analysis_data = match_dataframe_gold_analysis.copy()
match_dataframe_gold_analysis_data = match_dataframe_gold_analysis_data[match_dataframe_gold_analysis.GoldSpent != -1]
match_dataframe_gold_analysis_data = match_dataframe_gold_analysis_data[match_dataframe_gold_analysis.GoldRemaining != -1]
#(actually, the result would be the same wth just one line of code, since values seem to be missing concurrently)

display(match_dataframe_gold_analysis_data)

#Analyze the GoldDifference column
match_dataframe_gold_analysis_data['GoldDifference'].describe()

#The mean and 25, 50 percentiles are negative, while the 75 percentile shows a smaller positive value.
#This can be explained by the fact that gold is lost upon death, so let's explore how deaths in a game affect 
#the total gold.
#To that end, turn the difference into absolute values
match_dataframe_gold_analysis_data.GoldDifference = match_dataframe_gold_analysis_data.GoldDifference.abs()

#Check result

#Add the deaths column to the test dataframe
match_dataframe_gold_analysis_data['Deaths'] = match_dataframe['Deaths']
match_dataframe_gold_analysis_data['LostGoldPerDeath'] = match_dataframe_gold_analysis_data['GoldDifference'] / match_dataframe_gold_analysis_data['Deaths']
match_dataframe_gold_analysis_data['LostGoldPerDeath'].replace(to_replace = np.inf, value = 0, inplace = True)
match_dataframe_gold_analysis_data['LostGoldPerDeath'] = match_dataframe_gold_analysis_data['LostGoldPerDeath'].round().astype(int)

display(match_dataframe_gold_analysis_data)

#Look for trends in the difference
match_dataframe_gold_analysis_data.LostGoldPerDeath.describe()

#The median lost gold on death corresponds with my personal in-game experience. Even though it depends on your total
#net worth, it is a good estimate to fill in the missing data

#To get a better estimate of the total networth at the end of the game, I could take the average of gold values 
#generated by the sum in columns GoldSpent and GoldRemaining and the calculated gold from the GoldPerMin times 
#Duration columns. Since the data is missing for the former two most of the time, I can use the average lost gold
#on death times the number of deaths to make this estimate. 
#In other words, the formula would be (total_sum1 + total_sum2) / 2, and in this case it would be
#((GoldPerMin * Duration) + (GoldSpent + GoldRemaining)) / 2 for the known values and 
#(GoldPerMin * Duration) - Deaths * average_gold_loss / 2 if we work with absolute values
#or (GoldPerMin * Duration) + Deaths * average_gold_loss / 2 if we work with the initial data

#I will work with the original match_dataframe_gold_analysis dataframe:
lost_gold_on_death = 136
#I need the deaths column for claculations
match_dataframe_gold_analysis['Deaths'] = match_dataframe['Deaths']
total_gold = []

for i, row in match_dataframe_gold_analysis.iterrows():
    if row['GoldSpent'] == -1:
        total_gold.append(round(row['GoldPerMin'] * row['DurationMins'] - row['Deaths'] * lost_gold_on_death / 2))
    else:
        total_gold.append(round((row['GoldPerMin'] * row['DurationMins'] + row['GoldSpent'] + row['GoldRemaining']) / 2))
        
series = pd.Series(total_gold)
match_dataframe_gold_analysis['TotalGold'] = series.values

#Check results
display(match_dataframe_gold_analysis)

#Let's compare the results with pure Gold Per minute calculations
match_dataframe_gold_analysis['FinalDifference'] = (match_dataframe_gold_analysis['TotalGoldFromGPM'] - match_dataframe_gold_analysis['TotalGold']).abs()
display(match_dataframe_gold_analysis)

match_dataframe_gold_analysis.FinalDifference.describe()

#Seems good! Now we can drop these columns from the dataframe and just have the total gold left:
del match_dataframe['GoldSpent']
del match_dataframe['GoldRemaining']
del match_dataframe['ExpPerMin']

#Also dropping the ExpPerMin column, since it is not that useful anyway.
match_dataframe['TotalGold'] = match_dataframe_gold_analysis['TotalGoldFromGPM']
display(match_dataframe[['MatchID', 'Date', 'Duration', 'GoldPerMin', 'TotalGold']])

#Now all the missing values have been filled, and this dataframe can be used for in-game gold-related analysis!